In [1]:
import pandas as pd
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import re
from textblob import TextBlob

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack, csr_matrix
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import cross_val_score
import scipy.sparse as sparse
import string
import gc

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,make_scorer,roc_auc_score
from sklearn.metrics import f1_score,confusion_matrix
from sklearn import preprocessing 

from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
import seaborn as sns
%matplotlib inline
sns.set(style="darkgrid")
import numpy as np
from nltk.tokenize import TweetTokenizer


from sklearn.metrics.classification import classification_report, accuracy_score
from sklearn.cross_validation import cross_val_predict
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB


from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore")
from sklearn.utils import shuffle

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, precision_score

from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_selection import SelectKBest, chi2
import collections, itertools

In [2]:
df=pd.read_csv('./../clean_train_data.csv',index_col='id',encoding='utf-8')
del df['Unnamed: 0']
df['Tokenized1'].fillna('a',inplace=True)
'''def reduce_dimensions(train_dtm, y_train,test_dtm, k=15000):
    """Reduce dimensionality"""
    ch2 = SelectKBest(chi2, k)
    ch2.fit(train_dtm, y_train)
    train_dtm = ch2.transform(train_dtm)
    test_dtm = ch2.transform(test_dtm)
    return train_dtm, test_dtm'''

'def reduce_dimensions(train_dtm, y_train,test_dtm, k=15000):\n    """Reduce dimensionality"""\n    ch2 = SelectKBest(chi2, k)\n    ch2.fit(train_dtm, y_train)\n    train_dtm = ch2.transform(train_dtm)\n    test_dtm = ch2.transform(test_dtm)\n    return train_dtm, test_dtm'

# Preparing data for Model Execution

In [3]:
from nltk import word_tokenize
df = shuffle(df)
X=df.copy().drop(columns=['label'])
y=df['label']
df['Tokenized1'].fillna('a')

####tokenize-TfidfVectorizer
tknzr = TweetTokenizer()
vect_word = TfidfVectorizer(tokenizer=word_tokenize,max_features=1000, analyzer='word', stop_words='english', ngram_range=(1,2), dtype=np.float32) 
vect_word.fit(df.Tokenized1)
####transform
tr_tweetvec=vect_word.transform(df.Tokenized1)
####Horizontal stacking
train = df[['NoEx','NoHash','ListLength','No_HW0','No_HW1','Positive Score','Negative Score','LexicalPopularity','LCUCF','No_T']]
train_features = hstack([tr_tweetvec, csr_matrix(train.loc[train.index,])], 'csr')

# XGBoost

In [4]:
model1 = XGBClassifier()
model1.fit(train_features,y)
y_pred_train_xgb = cross_val_predict(model1,train_features,y,cv=5)
accuracy = accuracy_score(y, y_pred_train_xgb)
print(classification_report(y, y_pred_train_xgb))
print("Accuracy Train: %.2f%%" % (accuracy * 100.0))
print(f1_score(y, y_pred_train_xgb, average='weighted'))
print(confusion_matrix(y, y_pred_train_xgb))#50,62,239,1132

             precision    recall  f1-score   support

          0       0.97      0.99      0.98     29720
          1       0.83      0.54      0.65      2242

avg / total       0.96      0.96      0.96     31962

Accuracy Train: 95.95%
0.955435337939809
[[29466   254]
 [ 1041  1201]]


# Logistic Regression

In [5]:
model2 = LogisticRegression(penalty='l1',tol=.31)
model2.fit(train_features,y)
y_pred_train_LR = cross_val_predict(model2,train_features,y,cv=5)
accuracy = accuracy_score(y, y_pred_train_LR)
print(classification_report(y, y_pred_train_LR))
print("Accuracy Train: %.2f%%" % (accuracy * 100.0))
print(f1_score(y, y_pred_train_LR, average='weighted'))
print(confusion_matrix(y, y_pred_train_LR))#51,64,185,1094

             precision    recall  f1-score   support

          0       0.97      0.99      0.98     29720
          1       0.82      0.54      0.65      2242

avg / total       0.96      0.96      0.96     31962

Accuracy Train: 95.94%
0.955473623081853
[[29450   270]
 [ 1029  1213]]


# SVM with rbf kernel

In [ ]:
svmc = SVC(kernel="rbf")
svmc.fit(train_features,y)
preds_svmc = cross_val_predict(svmc,train_features,y,cv=5)
print(classification_report(y, preds_svmc)) 
print("ACCURACY Train::", accuracy_score(preds_svmc, y))
print(f1_score(y, preds_svmc, average='weighted'))
print(confusion_matrix(y, preds_svmc))

# Random Forest

In [6]:
rf = RandomForestClassifier(random_state=9)
rf.fit(train_features,y)
preds_rf = cross_val_predict(rf,train_features,y,cv=5)
print(classification_report(y, preds_rf))
print("ACCURACY Train::", accuracy_score(preds_rf, y))
print(f1_score(y, preds_rf, average='weighted'))
print(confusion_matrix(y, preds_rf))#57,69,185,974

             precision    recall  f1-score   support

          0       0.97      0.99      0.98     29720
          1       0.83      0.56      0.67      2242

avg / total       0.96      0.96      0.96     31962

ACCURACY Train:: 0.9610787810524999
0.9574835010540947
[[29467   253]
 [  991  1251]]


In [ ]:
df['pred']=preds_rf[:,]

In [ ]:
df.to_csv('./../clean_train_data1.csv',encoding='utf-8')

# XGBoost with parameter tuning

In [ ]:
param_grid1 = {"max_depth": [2, 3, 4,5,6,8,9],
               "min_child_weight": [4, 6, 7, 8,12],
               }
xgb = XGBClassifier(seed=9)

def myXGBoost(X_train, y_train, model, param_grid, KFold=3):
     
    acc_scorer = make_scorer(accuracy_score)
    grid_obj = RandomizedSearchCV(model, param_grid, scoring=acc_scorer,cv=KFold)
    grid_obj = grid_obj.fit(X_train, y_train)
    best_params=grid_obj.best_params_
    return grid_obj,best_params
xgbobj1,bestparam1_=myXGBoost(train_features, y, xgb, param_grid1, KFold=3)

In [ ]:
print(bestparam1_)
print(xgbobj1)
y_pred = cross_val_predict(xgbobj1,train_features,y,cv=5)
accuracy=accuracy_score(y,y_pred)

In [ ]:
auc_roc1 = roc_auc_score(y, y_pred)
print(classification_report(y, y_pred))
print(f1_score(y, y_pred, average='weighted'))
print(confusion_matrix(y, y_pred))

# Stacking

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
from mlxtend.classifier import StackingClassifier
import pandas as pd
import numpy as np
from sklearn.ensemble import VotingClassifier


model = [model2, rf]
# Write your code here
def stacking_clf(model, X_train, y_train):
    voting_clf_hard = VotingClassifier(estimators = [('Logistic Regression', model[0]),('Random Forest 1', model[1])],voting = 'soft')
    voting_clf_hard.fit(X_train, y_train)
    y_pred_hard = voting_clf_hard.predict(X_train)
    print(X_train.shape)
    y_pred_hard_DF=pd.DataFrame(y_pred_hard[:,])
    X_trainN = hstack([train_features, csr_matrix(y_pred_hard_DF)], 'csr')
    print(X_trainN.shape)
    stacking_clf1 = StackingClassifier(classifiers = model,meta_classifier = LogisticRegression(random_state=9))
    stacking_clf1.fit(X_trainN, y_train)
    preds_stacking = stacking_clf1.predict(X_trainN)##cross_val_predict(stacking_clf1,X_trainN, y_train,cv=5)
    print(classification_report(y_train, preds_stacking))
    print("ACCURACY Train::", accuracy_score(preds_stacking, y_train))
    print(f1_score(y_train, preds_stacking, average='weighted'))
    print(confusion_matrix(y_train, preds_stacking))
    return voting_clf_hard,stacking_clf1
voting_clf_hard,stacking_clf1=stacking_clf(model, train_features,y)#93,96,2,159

(31962, 1010)
(31962, 1011)
             precision    recall  f1-score   support

          0       0.99      1.00      1.00     29720
          1       0.99      0.93      0.96      2242

avg / total       0.99      0.99      0.99     31962

ACCURACY Train:: 0.99443088667793
0.9943378080349047
[[29709    11]
 [  167  2075]]


In [ ]:
def writeSubmission():
    
    df_test = pd.read_csv('./../clean_test_data.csv',encoding='utf-8')
    del df_test['Unnamed: 0']
    df_test['Tokenized1'].fillna('a',inplace=True)
    test_tweetvec=vect_word.transform(df_test.Tokenized1)
    test = df_test[['NoEx','NoHash','ListLength','No_HW0','No_HW1','Positive Score','Negative Score','LexicalPopularity','LCUCF','No_T']]
    test_features = hstack([test_tweetvec, csr_matrix(test.loc[test.index,])], 'csr')

    y_pred_hard_test = voting_clf_hard.predict(test_features)
    y_pred_hard_test_DF=pd.DataFrame(y_pred_hard_test[:,])
    X_testN = hstack([test_features, csr_matrix(y_pred_hard_test_DF)], 'csr')
    y_test_pred = stacking_clf1.predict(X_testN)

    df_sub = pd.DataFrame(y_test_pred,index=df_test['id'],columns=['label'])

    df_sub.to_csv('./../RFSubmission1.csv',encoding='utf-8')
writeSubmission()


# Writing Submission file

In [ ]:
def writeSubmission():
    df_test = pd.read_csv('./../clean_test_data.csv',encoding='utf-8')
    del df_test['Unnamed: 0']
    X_test=df_test.copy()
    test_tweetvec=vect_word.transform(df_test.Tokenized1)
    test = df_test[['NoEx','NoHash','ListLength','No_HW0','No_HW1','Positive Score','Negative Score','LexicalPopularity','LCUCF','No_T']]

    test_features = hstack([test_tweetvec, csr_matrix(test.loc[test.index,])], 'csr')
    y_test_pred=model2.predict(test_features)

    df_sub = pd.DataFrame(y_test_pred,index=df_test['id'],columns=['label'])

    df_sub.to_csv('./../RFSubmission1.csv',encoding='utf-8')
writeSubmission()